# TTfL and Rest API's

## RestAPI's and Python Requests

Transport for London (TfL) offers an easily accessible open data service. This information is conveniently obtainable through a RestAPI. API is short for Application Programming Interface, which is the designated interface for programming with a software. This can occur through various means, such as Grasshopper communicating with Rhino via an API. A RestAPI is more specific, as it involves communicating with a webserver through HTTP requests.

Essentially, this implies that you must create a specific text sequence that you can copy into your web browser. The browser then transmits this sequence to a webserver, which responds with the requested information.



Python provides several methods to send requests to a webserver, with the most popular approach being the utilization of a library known as [**"Requests"**](https://requests.readthedocs.io/en/latest/#).


![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Requests_Python_Logo.png/187px-Requests_Python_Logo.png)


## TfL

This is the link to the [general landing page](https://tfl.gov.uk/info-for/open-data-users/). Take your time and read a few of the posts, there are some interesting entries.

And this is the link to the [API Portal that](https://api-portal.tfl.gov.uk/api-details) helps you with building up the query.

You have the option to utilize the service anonymously, without any need for a subscription. Under this circumstance, you can make 50 requests per minute. However, if you subscribe (at no cost), you can send up to 500 requests per minute.

### TfL Example Air Quality
The following code demonstrates the typical setup with Requests. It's important to note that there are two options: one involving both a password and an ID, and another that doesn't require any authentication. For the sake of simplicity, we are using the code that doesn't require authentication.

In [ ]:
import requests

# Your ID and Key that you get from TfL
api_id = '...'
api_key = '...'

# Make a GET request to a specific endpoint
url = 'https://api.tfl.gov.uk/AirQuality/'

#OPTION 01 with password
#headers = {'App-ID': api_id, 'App-Key': api_key}
#response = requests.get(url, headers=headers)
#response = requests.get(url)

#OPTION 02 anoymous
response = requests.get(url)

# Check the status of the response
if response.status_code == 200:
    # Process the response data
    data = response.json()
    # Do something with the data
    print(data)
else:
    print(f'Request failed with status code {response.status_code}')

{'$id': '1', '$type': 'Tfl.Api.Presentation.Entities.LondonAirForecast, Tfl.Api.Presentation.Entities', 'updatePeriod': 'hourly', 'updateFrequency': '1', 'forecastURL': 'http://londonair.org.uk/forecast', 'disclaimerText': 'This forecast is intended to provide information on expected pollution levels in areas of significant public exposure. It may not apply in very specific locations close to unusually strong or short-lived local sources of pollution.', 'currentForecast': [{'$id': '2', '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities', 'forecastType': 'Current', 'forecastID': '42728', 'forecastBand': 'Low', 'forecastSummary': 'Low air pollution forecast valid from Monday 30 October to end of Monday 30 October GMT', 'nO2Band': 'Low', 'o3Band': 'Low', 'pM10Band': 'Low', 'pM25Band': 'Low', 'sO2Band': 'Low', 'forecastText': 'Mainly dry and bright on Monday with a chance of afternoon showers. &lt;br/&gt;&lt;br/&gt;An Atlantic air feed will aid dispersio

#### Parsing the result

We have now an object called `data` that holds the response from TfL. If you have a look a the code you will see this line:

`data = response.json()`

This implies that we need to instruct Python that the incoming response is in the form of a text file in JSON format. While there are various formats available, JSON is the most widely used. The basic structure is this:
```
{ "name": "Zophie",
  "isCat": true,
  "miceCaught": 0,
  "napsTaken": 37.5,
  "felineIQ": null}

```

Notice the similarity to a dict object:
```
{ "brand": "Ford",
  "model": "Mustang",
  "year": 1964 }
```
Below are examples how you can extract the results:

In [ ]:
result = data
result = data['$type']
result = data['currentForecast']
#result = data['currentForecast'][0]
#result = data['currentForecast'][0][forecastSummary]


result

In [ ]:
for enty in data['currentForecast']:
  print (enty['forecastSummary'])

Low air pollution forecast valid from Monday 30 October to end of Monday 30 October GMT
Low air pollution forecast valid from Tuesday 31 October to end of Tuesday 31 October GMT


### TfL Bikepoints London - Parsing and Export as csv

This excercise will examine the entirety of the bicycle locations in London. Given that the file is already substantial, it presents an excellent opportunity to analyze the response, construct a data frame, and store the data as a CSV. This process represents a common and standard workflow.

In [2]:
import requests
import pandas as pd

url = 'https://api.tfl.gov.uk/BikePoint/'
response = requests.get(url)

data = response.json()

### Sublime Text

![](https://upload.wikimedia.org/wikipedia/en/d/d2/Sublime_Text_3_logo.png)

The output file is quite large, and you might encounter difficulties if you aim to view it in your web browser. Fortunately, there's [Sublime Text](https://www.sublimetext.com/), a text editor like Notepad that can easily display sizable text files. Sublime supports external plugins that can be installed independently.

The best way to display json files would be with [Pretty JSON](https://github.com/dzhibas/SublimePrettyJson).

The installation goes in steps: After a fresh installation of sublime, you need to install the [Package Manager](https://packagecontrol.io/installation) first. This is a plug in that works as a plug in manager. Once installed, you press `ctrl+shift+p` and look for "Pretty JSON". Install that, select the entire body of the text, press `ctrl+shift+p` again and look for `pretty_json`.

This allows you to understand the structure of the file.

Belos is how to access the values of the first entry.

In [7]:
data[0]

id = data[0]['id']
commonName = data[0]['commonName']
lat = data[0]['lat']
lon = data[0]['lon']

print( "ID: " + str(id) + " / Name: " + str(commonName) + " / Latitude: " + str(lat) + " / Longitude: " + str(lon))

ID:BikePoints_1 / Name: River Street , Clerkenwell / Latitude: 51.529163 / Longitude: -0.10997


For all values it would be:

In [ ]:
for entry in data:
  id = entry['id']
  commonName = entry['commonName']
  lat = entry['lat']
  lon = entry['lon']
  print( "ID: " + str(id) + " / Name: " + str(commonName) + " / Latitude: " + str(lat) + " / Longitude: " + str(lon))

Great, but now we like to have it as a panda dataframe:

In [ ]:
df = pd.DataFrame(data, columns=['id','commonName','lat','lon'])

df

That was fast, it works so well because we do not access nesteld values and the column names remain as per the json file. All we have to do is to save the file as csv.

In [15]:
filepath = "/content/drive/MyDrive/Colab Notebooks/RC15 23 Excercises/Excercise 06 - Download Data from TfL/BikePointsLocation.csv"

df.to_csv(path_or_buf = filepath, header = ("ID", "commonName", "lat", "lon"))

And the comlpete script looks like this (less than ten lines of code):

In [16]:
import requests
import pandas as pd

url = 'https://api.tfl.gov.uk/BikePoint/'
response = requests.get(url)

data = response.json()

df = pd.DataFrame(data, columns=['id','commonName','lat','lon'])

filepath = "/content/drive/MyDrive/Colab Notebooks/RC15 23 Excercises/Excercise 06 - Download Data from TfL/BikePointsLocation.csv"

df.to_csv(path_or_buf = filepath, header = ("ID", "commonName", "lat", "lon"))

## Goodie : Traffic Webcams (Jam Cams) from TfL

The places api is interesting, it gives you all the TfL features within a certain radius of an point. The documentation is [here](https://api-portal.tfl.gov.uk/api-details#api=Place&operation=Place_MetaPlaceTypes).

These are the placetypes available:


```
["AreaForIntensification", "BikePoint", "Boroughs", "Cabwise", "CarPark",
 "CensusOutputAreas", "CensusSuperOutputAreas", "CentralActivityZone",
  "ChargeConnector", "ChargeStation", "CoachBay", "CoachPark", "CyclePark",
  "JamCam", "OnStreetMeteredBay", "OpportunityAreas", "OtherCoachParking",
  "OysterTicketShop", "RedLightAndSpeedCam", "RedLightCam", "SpeedCam",
  "TaxiRank", "VariableMessageSign", "Wards", "WaterfreightBridge",
  "WaterfreightDock", "WaterfreightJetty", "WaterfreightLock", "WaterfreightOther Access Point", "WaterfreightTunnel",
  "WaterfreightWharf"]
```
And with the help of the documentation, we can easy get the query code:

In [ ]:
import requests

url = "https://api.tfl.gov.uk/Place/?Lat=51.504593&Lon=-0.076641&radius=500&type=JamCam"

response = requests.get(url)

data = response.json()

data

Again, the best way is to have a look at the structure of the JSON file in sublime. We take the first object and we extract the info we want:

In [46]:
print ( "There are " + str(len(data["places"])) + " JamCams.")

entry = data["places"][2]

id = entry["id"]
commonName = entry["commonName"]
lat = entry["lat"]
lon = entry["lon"]
imageURL = entry["additionalProperties"][1]["value"]

print("ID"               + "\t\t"   + str(id) + "\n" \
      + "commonName"     + "\t"   + str(commonName) + "\n" \
      + "lat"            + "\t\t"   + str(lat) + "\n" \
      + "lon"            + "\t\t"   + str(lon) + "\n" \
      + "imageURL"       + "\t"   + str(imageURL))

There are 3 JamCams
ID		JamCams_00001.03540
commonName	Tooley St/Jamaica Rd
lat		51.5005
lon		-0.07445
imageURL	https://s3-eu-west-1.amazonaws.com/jamcams.tfl.gov.uk/00001.03540.jpg


and now all of them into a csv list

In [48]:
import pandas as pd

df = pd.DataFrame(columns=['ID', 'commonName', 'lat', 'lon', 'imageURL'])

for entry in data["places"]:
  id = entry["id"]
  commonName = entry["commonName"]
  lat = entry["lat"]
  lon = entry["lon"]
  imageURL = entry["additionalProperties"][1]["value"]
  df.loc[len(df)] = [id,commonName,lat,lon,imageURL]

df

,ID,commonName,lat,lon,imageURL
0,JamCams_00001.03506,Tooley St/Boss St,51.5020,-0.07681,https://s3-eu-west-1.amazonaws.com/jamcams.tfl...
1,JamCams_00001.03505,Tooley St/Abbots Lane,51.5039,-0.08136,https://s3-eu-west-1.amazonaws.com/jamcams.tfl...
2,JamCams_00001.03540,Tooley St/Jamaica Rd,51.5005,-0.07445,https://s3-eu-west-1.amazonaws.com/jamcams.tfl...
